# Corpus Lingustics Methods

- With `nltk`, we can easily implement quite a few corpus-linguistic methods
    - Concordance Analysis (Simple Words)
    - Frequency Lists
    - Collocations
    - Data Analysis with R
    - Concordance Analysis (Patterns, Constructions?)
        - Patterns on sentence strings
        - Patterns on sentence word-tag strings

## Preparing Corpus Data

In [1]:
import nltk
from nltk.corpus import brown
from nltk.text import Text
import pandas as pd

brown_text = Text(brown.words())

## Collocations

- Documentation [nltk.collocations](https://www.nltk.org/howto/collocations.html)

In [2]:
## Collocations based on Text
brown_text.collocation_list()[:10]
#brown_text.collocations()

from nltk.collocations import BigramAssocMeasures, BigramCollocationFinder

bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_words(brown.words())

In [3]:
## bigram collocations based on different association measures
finder.nbest(bigram_measures.likelihood_ratio,10)
finder.nbest(bigram_measures.pmi, 10)

[('$10,000-per-year', 'French-born'),
 ('$79.89', 'nothing-down'),
 ('$8.50', 'tab'),
 ("'low", 'nigras'),
 ('0.5-mv./m.', '50-percent'),
 ('0.78', 'mEq'),
 ('1,100', 'circumscriptions'),
 ('1,257,700', 'non-farm'),
 ('11-inch', 'headroom'),
 ('11-shot', 'hammerless')]

In [4]:
## Apply freq-based filers for bigram collocations
finder.apply_freq_filter(10)

## Apply word filer
from nltk.corpus import stopwords
stop_words_en = stopwords.words('english')
finder.apply_word_filter(lambda x: not x.isalpha())

finder.nbest(bigram_measures.likelihood_ratio, 10)
finder.nbest(bigram_measures.pmi, 10)

[('Hong', 'Kong'),
 ('Viet', 'Nam'),
 ('Pathet', 'Lao'),
 ('Simms', 'Purdew'),
 ('El', 'Paso'),
 ('Lo', 'Shu'),
 ('Internal', 'Revenue'),
 ('Puerto', 'Rico'),
 ('Saxon', 'Shore'),
 ('carbon', 'tetrachloride')]

In [5]:
## Create collocations based on tagged words
finder = BigramCollocationFinder.from_words(
    brown.tagged_words())
finder.apply_word_filter(lambda x: not x[0].isalpha())
finder.nbest(bigram_measures.pmi, 10)

[(('A', 'AT-NC'), ('charge', 'NN-NC')),
 (('A', 'FW-IN'), ('quibusdam', 'FW-WPO')),
 (('ACTH', 'NP'), ('Gel', 'NN-TL')),
 (('Abner', 'NP'), ('Haynes', 'NP')),
 (('Ace', 'NN-TL'), ('Driver', 'NN-TL')),
 (('Acoustical', 'JJ-HL'), ('interferometer', 'NN-HL')),
 (('Actively', 'RB-HL'), ('modernizing', 'VBG-HL')),
 (('Ad', 'FW-IN-TL'), ('Amicam', 'FW-NN-TL')),
 (('Adjusted', 'VBN-HL'), ('gross', 'JJ-HL')),
 (('Aerobacter', 'NP'), ('aerogenes', 'NP'))]

In [6]:
## Create collcoations based on tags only
finder = BigramCollocationFinder.from_words(
    t for w, t in brown.tagged_words(tagset='universal'))
finder.nbest(bigram_measures.pmi, 10)

[('X', 'X'),
 ('PRON', 'VERB'),
 ('PRT', 'VERB'),
 ('ADP', 'DET'),
 ('DET', 'ADJ'),
 ('ADJ', 'NOUN'),
 ('.', 'CONJ'),
 ('DET', 'NOUN'),
 ('VERB', 'PRT'),
 ('.', 'PRON')]

In [7]:
## Create collocations with intervneing words (gapped n-grams)
finder = BigramCollocationFinder.from_words(brown.words(), window_size=2)
finder.apply_word_filter(lambda x: not x.isalpha())
finder.apply_freq_filter(10)
finder.nbest(bigram_measures.pmi, 10)

[('Hong', 'Kong'),
 ('Viet', 'Nam'),
 ('Pathet', 'Lao'),
 ('Simms', 'Purdew'),
 ('El', 'Paso'),
 ('Lo', 'Shu'),
 ('Internal', 'Revenue'),
 ('Puerto', 'Rico'),
 ('Saxon', 'Shore'),
 ('carbon', 'tetrachloride')]

In [8]:
## Finders
scored = finder.score_ngrams(bigram_measures.raw_freq)
scored[:10]

[(('of', 'the'), 0.008288896237659233),
 (('in', 'the'), 0.004776126600941102),
 (('to', 'the'), 0.002950416468594341),
 (('on', 'the'), 0.0019781397047172215),
 (('and', 'the'), 0.001839489076741831),
 (('for', 'the'), 0.0015148226994329964),
 (('to', 'be'), 0.0014614292899021006),
 (('at', 'the'), 0.0012969431411859538),
 (('with', 'the'), 0.0012676628843464302),
 (('of', 'a'), 0.0012581898600748196)]

```{note}
How to get the document frequency of the bigrams???
```

In [9]:
unigram_freq = nltk.FreqDist(brown.words())
bigram_freq = nltk.FreqDist('_'.join(x) for x in nltk.bigrams(brown.words()))

In [10]:
unigram_freq_per_file = [nltk.FreqDist(words) 
                         for words in [brown.words(fileids=f) for f in brown.fileids()]]
bigram_freq_per_file = [nltk.FreqDist('_'.join(x) for x in nltk.bigrams(words))
                         for words in [brown.words(fileids=f) for f in brown.fileids()]]

In [11]:
## Function to get unigram dispersion
def createUnigramDipsersionDist(uni_freq, uni_freq_per_file):
    len(uni_freq_per_file)
    unigram_dispersion = {}

    for fid in uni_freq_per_file:
        for w, f in fid.items():
            if w in unigram_dispersion:
                unigram_dispersion[w] += 1
            else:
                unigram_dispersion[w] = 1
    return(unigram_dispersion)


In [12]:
unigram_dispersion = createUnigramDipsersionDist(unigram_freq, unigram_freq_per_file)
# Dictionary cannot be sliced/subset
# Get the items() and convert to list for subsetting
list(unigram_dispersion.items())[:20]

[('The', 500),
 ('Fulton', 3),
 ('County', 45),
 ('Grand', 17),
 ('Jury', 4),
 ('said', 314),
 ('Friday', 34),
 ('an', 498),
 ('investigation', 34),
 ('of', 500),
 ("Atlanta's", 2),
 ('recent', 114),
 ('primary', 59),
 ('election', 28),
 ('produced', 66),
 ('``', 462),
 ('no', 455),
 ('evidence', 119),
 ("''", 463),
 ('that', 500)]

In [13]:
#dict(sorted(bigram_freq.items()[:3]))
list(bigram_freq.items())[:20]

[('The_Fulton', 1),
 ('Fulton_County', 6),
 ('County_Grand', 1),
 ('Grand_Jury', 2),
 ('Jury_said', 1),
 ('said_Friday', 4),
 ('Friday_an', 1),
 ('an_investigation', 7),
 ('investigation_of', 15),
 ("of_Atlanta's", 1),
 ("Atlanta's_recent", 1),
 ('recent_primary', 1),
 ('primary_election', 2),
 ('election_produced', 1),
 ('produced_``', 1),
 ('``_no', 6),
 ('no_evidence', 14),
 ("evidence_''", 3),
 ("''_that", 16),
 ('that_any', 31)]

In [14]:
bigram_dispersion = createUnigramDipsersionDist(bigram_freq, bigram_freq_per_file)
list(bigram_dispersion.items())[:20]

[('The_Fulton', 1),
 ('Fulton_County', 1),
 ('County_Grand', 1),
 ('Grand_Jury', 2),
 ('Jury_said', 1),
 ('said_Friday', 3),
 ('Friday_an', 1),
 ('an_investigation', 7),
 ('investigation_of', 14),
 ("of_Atlanta's", 1),
 ("Atlanta's_recent", 1),
 ('recent_primary', 1),
 ('primary_election', 2),
 ('election_produced', 1),
 ('produced_``', 1),
 ('``_no', 6),
 ('no_evidence', 12),
 ("evidence_''", 3),
 ("''_that", 16),
 ('that_any', 30)]

In [15]:
type(unigram_freq)
type(unigram_dispersion)

dict

## Delta P

In [16]:
## Inherit BigramAssocMeasures
class AugmentedBigramAssocMeasures(BigramAssocMeasures):
    @classmethod
    def dp_fwd(self, *marginals):
        """Scores bigrams using DP forward
        This may be shown with respect to a contingency table::

                w1    ~w1
             ------ ------
         w2 | n_ii | n_oi | = n_xi
             ------ ------
        ~w2 | n_io | n_oo |
             ------ ------
             = n_ix        TOTAL = n_xx
        """
        
        n_ii, n_io, n_oi, n_oo = self._contingency(*marginals)

        return (n_ii/(n_ii+n_io)) - (n_oi/(n_oi+n_oo))

    @classmethod
    def dp_bwd(self, *marginals):
        """Scores bigrams using DP backward
        This may be shown with respect to a contingency table::

                w1    ~w1
             ------ ------
         w2 | n_ii | n_oi | = n_xi
             ------ ------
        ~w2 | n_io | n_oo |
             ------ ------
             = n_ix        TOTAL = n_xx
        """
        
        n_ii, n_io, n_oi, n_oo = self._contingency(*marginals)

        return (n_ii/(n_ii+n_oi)) - (n_io/(n_io+n_oo))

In [17]:
bigram_measures = AugmentedBigramAssocMeasures()
finder = BigramCollocationFinder.from_words(brown.words())

In [18]:
finder.apply_freq_filter(10)

In [19]:
bigrams_dpfwd = finder.score_ngrams(bigram_measures.dp_fwd)
bigrams_dpfwd[:10]

[(('Hong', 'Kong'), 1.0),
 (('Lo', 'Shu'), 1.0),
 (('Viet', 'Nam'), 0.9999974164190018),
 (('Puerto', 'Rico'), 0.9999974164012019),
 (('Los', 'Angeles'), 0.9999965551244676),
 (('Simms', 'Purdew'), 0.9999956940353778),
 (('El', 'Paso'), 0.9999913880855886),
 (('Blue', 'Throat'), 0.9999698582736025),
 (('Du', "Pont's"), 0.999955217852231),
 (('sweet', 'clover'), 0.9999552178136648)]

In [20]:
bigrams_dpbwd = finder.score_ngrams(bigram_measures.dp_bwd)
bigrams_dpbwd[:10]

[(('Hong', 'Kong'), 1.0),
 (('Lo', 'Shu'), 1.0),
 (('Pathet', 'Lao'), 0.9999939716599121),
 (('Export-Import', 'Bank'), 0.9999870820838838),
 (("Drug's", 'chemical'), 0.999963829803725),
 (('text-form', 'list'), 0.9998966570270638),
 (('Inquirer', ':'), 0.9984653528870225),
 (('AP', ')'), 0.9978883513122019),
 (('accordance', 'with'), 0.9939784975869208),
 (('compatible', 'with'), 0.9939750735461291)]

## Concordance

In [21]:
## Simple Concordances
brown_text.concordance('American', width=79, lines = 5)

Displaying 5 of 569 matches:
will deliver tomorrow night to the American people over nationwide television 
ocial security taxes on 70 million American workers would be raised to pay the
o retired as vice president of the American Screw Co. in 1955 said , `` Both p
wice elected overwhelmingly by the American people as president of the United 
n example : Last month in Ghana an American missionary discovered when he came


In [22]:
#nltk.app.concordance()

In [23]:
## Regular Expression Concordances
import re
sents = [' '.join(s) for s in brown.sents()]
regex_1 = r'(is|was) \w+ing'
targets = [sent for sent in sents[:100] if re.search(regex_1, sent)]
targets[0]
#if targets:
#    for match in targets:
#        print(match.strip())

"The City Purchasing Department , the jury said , `` is lacking in experienced clerical personnel as a result of city personnel policies '' ."

## Frequency List

In [24]:
## word frequencies
brown_fd_words = nltk.FreqDist(brown.words())
brown_fd_words.most_common(10)

[('the', 62713),
 (',', 58334),
 ('.', 49346),
 ('of', 36080),
 ('and', 27915),
 ('to', 25732),
 ('a', 21881),
 ('in', 19536),
 ('that', 10237),
 ('is', 10011)]

In [25]:
## nouns freq
brown_fd_nouns = nltk.FreqDist([w.lower() for w,t in brown.tagged_words() 
                                 if any (noun_tag in t for noun_tag in ['NP','NN'])])
brown_fd_nouns.most_common(10)

brown_fd_nouns_df = pd.DataFrame(brown_fd_nouns.items(), columns=['word','freq'])

Sort the data frame:

In [26]:
brown_fd_nouns_df[brown_fd_nouns_df['freq']>100].sort_values(["freq","word"],ascending=[False,True])

,word,freq
243,time,1597
174,man,1203
5114,af,995
248,years,949
779,way,899
...,...,...
1030,events,101
204,james,101
1723,officer,101
272,test,101


```{note}
We can also pass the data frame to R for data exploration.
```

In [27]:
%load_ext rpy2.ipython

In [28]:
%%R -i brown_fd_nouns_df

library(dplyr)
brown_fd_nouns_df %>%
filter(freq > 100) %>%
arrange(desc(freq), word) %>% 
head(50)

R[write to console]: 
Attaching package: ‘dplyr’




R[write to console]: The following objects are masked from ‘package:stats’:

    filter, lag




R[write to console]: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




        word

 freq

...

       power

  340


595 

 development

  333

## Conditional Frequency List


In [29]:
## Word by POS Frequency Distribution

brown_news_tagged_words = brown.tagged_words(categories='news', tagset='universal')
brown_news_cfd = nltk.ConditionalFreqDist(brown_news_tagged_words)
brown_news_cfd['yield']

FreqDist({'NOUN': 5, 'VERB': 1})

In [30]:
## POS by Word Frequency Distribution
brown_news_cfd2 = nltk.ConditionalFreqDist([(t, w) for (w, t) in brown_news_tagged_words])
brown_news_cfd2['VERB'].most_common(10)

[('is', 732),
 ('was', 717),
 ('be', 526),
 ('said', 402),
 ('will', 388),
 ('are', 328),
 ('has', 300),
 ('had', 279),
 ('have', 265),
 ('were', 252)]

In [31]:
## Word by Genre Frequency Distribution
brown_genre_cfd = nltk.ConditionalFreqDist(
    (word, genre)
    for genre in brown.categories()
    for word in brown.words(categories=genre)
)

In [32]:
brown_genre_cfd.conditions()[:50]
brown_genre_cfd['mysterious']

FreqDist({'belles_lettres': 6, 'fiction': 4, 'lore': 3, 'religion': 3, 'romance': 3, 'learned': 2, 'reviews': 2, 'adventure': 1, 'humor': 1, 'science_fiction': 1})

In [33]:
print(sorted(brown_genre_cfd['mysterious'].items(),key=lambda x:x[1],reverse=True)) # with freq

[('belles_lettres', 6), ('fiction', 4), ('lore', 3), ('religion', 3), ('romance', 3), ('learned', 2), ('reviews', 2), ('adventure', 1), ('humor', 1), ('science_fiction', 1)]


In [34]:
## Genre by Word Frequency Distribution
brown_genre_cdf2 = nltk.ConditionalFreqDist(
    (genre, word)
    for genre in brown.categories()
    for word in brown.words(categories=genre)
)

In [35]:
## Genre by Word Frequency Distribution
brown_genre_cdf2 = nltk.ConditionalFreqDist(
    (genre, word)
    for genre in brown.categories()
    for word in brown.words(categories=genre)
)



In [36]:

top_n_word = [word for (word, freq) in brown_fd_words.most_common(20) if word[0].isalpha()]

brown_genre_cdf2.tabulate(conditions=['adventure','editorial','fiction'],
                         samples=top_n_word[:10])

           the   of  and   to    a   in that   is  was  for 
adventure 3370 1322 1622 1309 1354  847  494   98  914  331 
editorial 3508 1976 1302 1554 1095 1001  578  744  308  509 
  fiction 3423 1419 1696 1489 1281  916  530  144 1082  392 


In [37]:
top_n_word2 = [word for (word, tag) in brown.tagged_words(tagset='universal') 
               if tag.startswith('NOUN')]
top_n_word2_fd = nltk.FreqDist(top_n_word2).most_common(10)

In [38]:
print(top_n_word2_fd)
brown_genre_cdf2.tabulate(conditions=['adventure','editorial','fiction'],
                         samples=[w for (w, f) in top_n_word2_fd])

[('time', 1555), ('man', 1148), ('Af', 994), ('years', 942), ('way', 883), ('Mr.', 844), ('people', 809), ('men', 736), ('world', 684), ('life', 676)]
            time    man     Af  years    way    Mr. people    men  world   life 
adventure    127    165      0     32     65     22     24     81     15     29 
editorial     72     56      0     63     43    110     75     38     66     35 
  fiction     99    111      0     44     62     39     39     72     24     44 
